In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
year = 2020
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
html = urlopen(url)
soup = BeautifulSoup(html)

In [3]:
#Use findALL() to get the column headers
soup.findAll('tr', limit=2)
#Use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
#Exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]
#headers

In [4]:
#Avoid the first header row
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

In [5]:
stats_2020 = pd.DataFrame(player_stats, columns = headers)

In [6]:
stats_2020.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [7]:
stats_2020=stats_2020[['Player','Tm', 'eFG%','3P',
       '3PA', '3P%', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', 'FT', 'FTA', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','FT%','Pos', 'Age','G', 'GS','MP']]

In [8]:
stats_2020.head(2)

,Player,Tm,eFG%,3P,3PA,3P%,FG,FGA,FG%,2P,...,BLK,TOV,PF,PTS,FT%,Pos,Age,G,GS,MP
0,Steven Adams,OKC,.593,0.0,0.0,.333,4.5,7.6,.592,4.5,...,1.1,1.5,1.9,10.9,.582,C,26,63,63,26.7
1,Bam Adebayo,MIA,.558,0.0,0.2,.143,6.1,11.0,.557,6.1,...,1.3,2.8,2.5,15.9,.691,PF,22,72,72,33.6


In [9]:
stats_2020.rename(columns = {'3PA':'Attempts_3PT'}, inplace = True)
stats_2020.rename(columns = {'3P':'Made_3PT'}, inplace = True)
stats_2020.rename(columns = {'3P%':'Success_3PT'}, inplace = True)
stats_2020.rename(columns = {'eFG%':'eFG_Pct'}, inplace = True)
stats_2020.rename(columns = {'FG%':'FG_Pct'}, inplace = True)

stats_2020=stats_2020.dropna(subset=['Success_3PT'])
stats_2020=stats_2020.dropna(subset=['eFG_Pct'])
stats_2020=stats_2020.dropna(subset=['FG_Pct'])

stats_2020['Attempts_3PT']=stats_2020['Attempts_3PT'].astype(float)
stats_2020['Made_3PT']=stats_2020['Made_3PT'].astype(float)
stats_2020['Attempts_3PT']=stats_2020['Attempts_3PT'].astype(float)
stats_2020['FGA']=stats_2020['FGA'].astype(float)
#stats_2020['Success_3PT']=stats_2020['Success_3PT'].astype(float)
#stats_2020['eFG_Pct']=stats_2020['eFG_Pct'].astype(float)
#stats_2020['FG_Pct']=stats_2020['FG_Pct'].astype(float)

In [10]:
#Add some new columns with sample data, which we will change in a moment to reflect the actual values.
stats_2020.insert(3, 'Pct_3PA', .1, True)
stats_2020['Pct_3PA']=stats_2020['Attempts_3PT']/stats_2020['FGA']

In [11]:
stats_2020.head(2)

,Player,Tm,eFG_Pct,Pct_3PA,Made_3PT,Attempts_3PT,Success_3PT,FG,FGA,FG_Pct,...,BLK,TOV,PF,PTS,FT%,Pos,Age,G,GS,MP
0,Steven Adams,OKC,.593,0.000000,0.0,0.0,.333,4.5,7.6,.592,...,1.1,1.5,1.9,10.9,.582,C,26,63,63,26.7
1,Bam Adebayo,MIA,.558,0.018182,0.0,0.2,.143,6.1,11.0,.557,...,1.3,2.8,2.5,15.9,.691,PF,22,72,72,33.6


In [12]:
stats_2020['G']=stats_2020['G'].astype(int)
stats_2020['GS']=stats_2020['GS'].astype(int)
stats_2020['Age']=stats_2020['Age'].astype(int)

In [13]:
stats_2020

,Player,Tm,eFG_Pct,Pct_3PA,Made_3PT,Attempts_3PT,Success_3PT,FG,FGA,FG_Pct,...,BLK,TOV,PF,PTS,FT%,Pos,Age,G,GS,MP
0,Steven Adams,OKC,.593,0.000000,0.0,0.0,.333,4.5,7.6,.592,...,1.1,1.5,1.9,10.9,.582,C,26,63,63,26.7
1,Bam Adebayo,MIA,.558,0.018182,0.0,0.2,.143,6.1,11.0,.557,...,1.3,2.8,2.5,15.9,.691,PF,22,72,72,33.6
2,LaMarcus Aldridge,SAS,.532,0.200000,1.2,3.0,.389,7.4,15.0,.493,...,1.6,1.4,2.4,18.9,.827,C,34,53,53,33.1
3,Kyle Alexander,MIA,.500,0.000000,0.0,0.0,,0.5,1.0,.500,...,0.0,0.5,0.5,1.0,,C,23,2,0,6.5
4,Nickeil Alexander-Walker,NOP,.455,0.491228,1.0,2.8,.346,2.1,5.7,.368,...,0.2,1.1,1.2,5.7,.676,SG,21,47,1,12.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,Trae Young,ATL,.519,0.456731,3.4,9.5,.361,9.1,20.8,.437,...,0.1,4.8,1.7,29.6,.860,PG,21,60,60,35.3
673,Cody Zeller,CHO,.543,0.156627,0.3,1.3,.240,4.3,8.3,.524,...,0.4,1.3,2.4,11.1,.682,C,27,58,39,23.1
674,Tyler Zeller,SAS,.250,0.000000,0.0,0.0,,0.5,2.0,.250,...,0.0,0.0,0.0,1.0,,C,30,2,0,2.0
675,Ante Žižić,CLE,.569,0.000000,0.0,0.0,,1.9,3.3,.569,...,0.2,0.5,1.2,4.4,.737,C,23,22,0,10.0


In [14]:
stats_2020['MP']=stats_2020['MP'].astype(float)

In [15]:
stats_2020.MP.describe()

count    651.000000
mean      19.179724
std        9.060972
min        1.000000
25%       12.000000
50%       18.500000
75%       26.700000
max       37.500000
Name: MP, dtype: float64

In [16]:
stats_2020.sort_values(['eFG_Pct'], ascending=False)

,Player,Tm,eFG_Pct,Pct_3PA,Made_3PT,Attempts_3PT,Success_3PT,FG,FGA,FG_Pct,...,BLK,TOV,PF,PTS,FT%,Pos,Age,G,GS,MP
13,Kostas Antetokounmpo,LAL,1.000,0.000000,0.0,0.0,,0.6,0.6,1.000,...,0.0,0.2,0.4,1.4,.500,PF,22,5,0,4.0
625,Noah Vonleh,DEN,.917,0.111111,0.1,0.1,1.000,0.7,0.9,.833,...,0.0,0.3,0.6,1.9,.500,C,24,7,0,4.3
209,Tacko Fall,BOS,.786,0.000000,0.0,0.0,,1.6,2.0,.786,...,0.6,0.4,1.0,3.3,.333,C,24,7,0,4.7
263,Donta Hall,BRK,.778,0.000000,0.0,0.0,,2.8,3.6,.778,...,1.0,0.6,2.8,6.6,.417,C,22,5,0,17.0
122,Tyson Chandler,HOU,.778,0.000000,0.0,0.0,,0.5,0.7,.778,...,0.3,0.3,1.2,1.3,.462,C,37,26,5,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Jacob Evans,MIN,.000,1.000000,0.0,0.5,.000,0.0,0.5,.000,...,0.0,0.0,0.0,0.0,,SG,22,2,0,2.0
477,Zach Norvell,LAL,.000,0.000000,0.0,0.0,,0.0,0.5,.000,...,0.0,0.0,0.0,0.0,,SG,22,2,0,2.5
310,William Howard,HOU,.000,1.000000,0.0,2.0,.000,0.0,2.0,.000,...,0.0,0.5,0.5,0.0,,PF,26,2,0,6.5
67,Marques Bolden,CLE,,NaN,0.0,0.0,,0.0,0.0,,...,0.0,0.0,1.0,0.0,,C,21,1,0,3.0


In [17]:
stats_2020=stats_2020.dropna(subset=['eFG_Pct'])

In [18]:
stats_2020.sort_values(['eFG_Pct'], ascending=False)

,Player,Tm,eFG_Pct,Pct_3PA,Made_3PT,Attempts_3PT,Success_3PT,FG,FGA,FG_Pct,...,BLK,TOV,PF,PTS,FT%,Pos,Age,G,GS,MP
13,Kostas Antetokounmpo,LAL,1.000,0.000000,0.0,0.0,,0.6,0.6,1.000,...,0.0,0.2,0.4,1.4,.500,PF,22,5,0,4.0
625,Noah Vonleh,DEN,.917,0.111111,0.1,0.1,1.000,0.7,0.9,.833,...,0.0,0.3,0.6,1.9,.500,C,24,7,0,4.3
209,Tacko Fall,BOS,.786,0.000000,0.0,0.0,,1.6,2.0,.786,...,0.6,0.4,1.0,3.3,.333,C,24,7,0,4.7
263,Donta Hall,BRK,.778,0.000000,0.0,0.0,,2.8,3.6,.778,...,1.0,0.6,2.8,6.6,.417,C,22,5,0,17.0
122,Tyson Chandler,HOU,.778,0.000000,0.0,0.0,,0.5,0.7,.778,...,0.3,0.3,1.2,1.3,.462,C,37,26,5,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Jacob Evans,MIN,.000,1.000000,0.0,0.5,.000,0.0,0.5,.000,...,0.0,0.0,0.0,0.0,,SG,22,2,0,2.0
477,Zach Norvell,LAL,.000,0.000000,0.0,0.0,,0.0,0.5,.000,...,0.0,0.0,0.0,0.0,,SG,22,2,0,2.5
310,William Howard,HOU,.000,1.000000,0.0,2.0,.000,0.0,2.0,.000,...,0.0,0.5,0.5,0.0,,PF,26,2,0,6.5
67,Marques Bolden,CLE,,NaN,0.0,0.0,,0.0,0.0,,...,0.0,0.0,1.0,0.0,,C,21,1,0,3.0


In [19]:
#Should drop FGA values of 0.0.
#Should do the same for 0.0 Attempts_3PT values.

In [20]:
stats_2020 = stats_2020[stats_2020.FGA != 0]
stats_2020 = stats_2020[stats_2020.Attempts_3PT != 0]
stats_2020 = stats_2020[stats_2020.MP >= 20]
stats_2020

,Player,Tm,eFG_Pct,Pct_3PA,Made_3PT,Attempts_3PT,Success_3PT,FG,FGA,FG_Pct,...,BLK,TOV,PF,PTS,FT%,Pos,Age,G,GS,MP
1,Bam Adebayo,MIA,.558,0.018182,0.0,0.2,.143,6.1,11.0,.557,...,1.3,2.8,2.5,15.9,.691,PF,22,72,72,33.6
2,LaMarcus Aldridge,SAS,.532,0.200000,1.2,3.0,.389,7.4,15.0,.493,...,1.6,1.4,2.4,18.9,.827,C,34,53,53,33.1
6,Jarrett Allen,BRK,.649,0.015152,0.0,0.1,.000,4.3,6.6,.649,...,1.3,1.1,2.3,11.1,.633,C,21,70,64,26.5
8,Al-Farouq Aminu,ORL,.343,0.416667,0.5,2.0,.250,1.4,4.8,.291,...,0.4,0.9,1.5,4.3,.655,PF,29,18,2,21.1
12,Giannis Antetokounmpo,MIL,.589,0.238579,1.4,4.7,.304,10.9,19.7,.553,...,1.0,3.7,3.1,29.5,.633,PF,25,63,63,30.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,Christian Wood,DET,.620,0.280488,0.9,2.3,.386,4.6,8.2,.567,...,0.9,1.4,1.6,13.1,.744,PF,24,62,12,21.4
669,Delon Wright,DAL,.519,0.303571,0.6,1.7,.370,2.6,5.6,.462,...,0.3,1.0,1.3,6.9,.770,PG,27,73,5,21.5
671,Thaddeus Young,CHI,.513,0.372340,1.2,3.5,.356,4.2,9.4,.448,...,0.4,1.6,2.1,10.3,.583,PF,31,64,16,24.9
672,Trae Young,ATL,.519,0.456731,3.4,9.5,.361,9.1,20.8,.437,...,0.1,4.8,1.7,29.6,.860,PG,21,60,60,35.3


In [21]:
stats_2020['Success_3PT']=stats_2020['Success_3PT'].astype(float)
stats_2020['eFG_Pct']=stats_2020['eFG_Pct'].astype(float)
stats_2020['FG_Pct']=stats_2020['FG_Pct'].astype(float)

In [22]:
stats_2020.eFG_Pct.describe()

count    284.000000
mean       0.525556
std        0.048828
min        0.343000
25%        0.493000
50%        0.524000
75%        0.552500
max        0.667000
Name: eFG_Pct, dtype: float64

In [23]:
stats_2020.Pct_3PA.describe()

count    284.000000
mean       0.412002
std        0.173224
min        0.008772
25%        0.311437
50%        0.415109
75%        0.518477
max        0.882979
Name: Pct_3PA, dtype: float64

In [24]:
stats_2020.insert(4, 'Flag_3', 0, True)
stats_2020.loc[(stats_2020['Pct_3PA']>=.25), 'Flag_3']=1

In [25]:
stats_2020.Flag_3.describe()

count    284.000000
mean       0.823944
std        0.381540
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Flag_3, dtype: float64

In [26]:
stats_2020.Made_3PT.describe()

count    284.000000
mean       1.556690
std        0.858829
min        0.000000
25%        1.000000
50%        1.500000
75%        2.100000
max        4.400000
Name: Made_3PT, dtype: float64

In [27]:
stats_2020.Attempts_3PT.describe()

count    284.000000
mean       4.325000
std        2.163733
min        0.100000
25%        3.000000
50%        4.050000
75%        5.700000
max       12.400000
Name: Attempts_3PT, dtype: float64

In [46]:
stats_2020['Success_3PT']=stats_2020['Success_3PT'].astype(float)

In [47]:
stats_2020.Success_3PT.describe()

count    284.000000
mean       0.345599
std        0.068715
min        0.000000
25%        0.321750
50%        0.356000
75%        0.386250
max        0.571000
Name: Success_3PT, dtype: float64

In [28]:
year = 1995
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
html = urlopen(url)
soup = BeautifulSoup(html)

In [29]:
#Use findALL() to get the column headers
soup.findAll('tr', limit=2)
#Use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
#Exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]
#headers

In [30]:
#Avoid the first header row
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

In [31]:
stats_1995 = pd.DataFrame(player_stats, columns = headers)

In [32]:
stats_1995=stats_1995[['Player', 'Tm','eFG%','3P',
       '3PA', '3P%', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', 'FT', 'FTA', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','FT%','Pos', 'Age','G', 'GS','MP']]

In [33]:
stats_1995.rename(columns = {'3PA':'Attempts_3PT'}, inplace = True)
stats_1995.rename(columns = {'3P':'Made_3PT'}, inplace = True)
stats_1995.rename(columns = {'3P%':'Success_3PT'}, inplace = True)
stats_1995.rename(columns = {'eFG%':'eFG_Pct'}, inplace = True)
stats_1995.rename(columns = {'FG%':'FG_Pct'}, inplace = True)

stats_1995=stats_1995.dropna(subset=['Success_3PT'])
stats_1995=stats_1995.dropna(subset=['eFG_Pct'])
stats_1995=stats_1995.dropna(subset=['FG_Pct'])

stats_1995['Attempts_3PT']=stats_1995['Attempts_3PT'].astype(float)
stats_1995['Made_3PT']=stats_1995['Made_3PT'].astype(float)
stats_1995['Attempts_3PT']=stats_1995['Attempts_3PT'].astype(float)
stats_1995['MP']=stats_1995['MP'].astype(float)
stats_1995['FGA']=stats_1995['FGA'].astype(float)
stats_1995=stats_1995.dropna(subset=['eFG_Pct'])
stats_1995 = stats_1995[stats_1995.eFG_Pct != 0]

In [34]:
stats_1995 = stats_1995[stats_1995.FGA != 0]
stats_1995 = stats_1995[stats_1995.Attempts_3PT != 0]
stats_1995 = stats_1995[stats_1995.MP >= 20]

In [35]:
#Add some new columns with sample data, which we will change in a moment to reflect the actual values.
stats_1995.insert(3, 'Pct_3PA', .1, True)
stats_1995['Pct_3PA']=stats_1995['Attempts_3PT']/stats_1995['FGA']

In [36]:
stats_1995['G']=stats_1995['G'].astype(int)
stats_1995['GS']=stats_1995['GS'].astype(int)
stats_1995['Age']=stats_1995['Age'].astype(int)
stats_1995['MP']=stats_1995['MP'].astype(float)
stats_1995['eFG_Pct']=stats_1995['eFG_Pct'].astype(float)

In [37]:
stats_1995.insert(4, 'Flag_3', 0, True)
stats_1995.loc[(stats_1995['Pct_3PA']>=.25), 'Flag_3']=1
stats_1995

,Player,Tm,eFG_Pct,Pct_3PA,Flag_3,Made_3PT,Attempts_3PT,Success_3PT,FG,FGA,...,BLK,TOV,PF,PTS,FT%,Pos,Age,G,GS,MP
3,Mahmoud Abdul-Rauf,DEN,0.511,0.210145,0,1.1,2.9,.386,6.5,13.8,...,0.1,1.6,1.7,16.0,.885,PG,25,73,43,28.5
5,Rafael Addison,DET,0.497,0.148649,0,0.3,1.1,.289,3.5,7.4,...,0.3,1.0,3.0,8.3,.747,SF,30,79,16,22.5
7,Victor Alexander,GSW,0.521,0.056180,0,0.1,0.5,.240,4.6,8.9,...,0.6,1.5,2.9,10.0,.600,C,25,50,29,24.7
10,Kenny Anderson,NJN,0.446,0.286713,1,1.3,4.1,.330,5.7,14.3,...,0.2,3.1,2.6,17.6,.841,PG,24,72,70,37.3
11,Nick Anderson,ORL,0.573,0.471074,1,2.4,5.7,.415,5.8,12.1,...,0.3,1.9,1.6,15.8,.704,SG,27,76,76,34.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Walt Williams,SAC,0.497,0.292308,1,1.3,3.8,.348,5.8,13.0,...,0.8,3.2,3.4,16.4,.731,SF,24,77,77,35.6
459,Kevin Willis,TOT,0.467,0.013245,0,0.0,0.2,.200,7.1,15.1,...,0.5,2.4,3.2,17.2,.690,PF,32,67,63,35.7
460,Kevin Willis,ATL,0.390,0.024390,0,0.0,0.5,.000,8.0,20.5,...,1.5,3.5,3.5,21.0,.667,PF,32,2,2,44.5
461,Kevin Willis,MIA,0.471,0.013333,0,0.0,0.2,.214,7.0,15.0,...,0.5,2.4,3.2,17.1,.691,PF,32,65,61,35.4


In [38]:
stats_1995.Flag_3.describe()

count    185.000000
mean       0.443243
std        0.498116
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Flag_3, dtype: float64

In [39]:
stats_1995.eFG_Pct.describe()

count    185.000000
mean       0.500022
std        0.042399
min        0.390000
25%        0.472000
50%        0.497000
75%        0.528000
max        0.617000
Name: eFG_Pct, dtype: float64

Of all players to average at least 20 minutes per game (MP), 44% took at least a quarter of their shots from 3-point range in 1995. In 2020, 82% took at least a quarter of their shots from 3-point range, nearly double.

We can also see that players are more efficient, with average effective field goal percentage (eFG_Pct) at 50.0% in 1995 but 52.6% in 2020.

In [42]:
stats_1995['Success_3PT']=stats_1995['Success_3PT'].astype(float)

In [43]:
stats_1995.Success_3PT.describe()

count    185.000000
mean       0.307341
std        0.113922
min        0.000000
25%        0.268000
50%        0.330000
75%        0.386000
max        0.524000
Name: Success_3PT, dtype: float64

In [44]:
stats_1995.Made_3PT.describe()

count    185.000000
mean       0.852973
std        0.729580
min        0.000000
25%        0.100000
50%        0.800000
75%        1.300000
max        2.700000
Name: Made_3PT, dtype: float64

In [45]:
stats_1995.Attempts_3PT.describe()

count    185.000000
mean       2.404865
std        1.908826
min        0.100000
25%        0.500000
50%        2.200000
75%        3.600000
max        7.600000
Name: Attempts_3PT, dtype: float64

In [40]:
stats_1995.sort_values(['eFG_Pct'], ascending=False)

,Player,Tm,eFG_Pct,Pct_3PA,Flag_3,Made_3PT,Attempts_3PT,Success_3PT,FG,FGA,...,BLK,TOV,PF,PTS,FT%,Pos,Age,G,GS,MP
220,Steve Kerr,CHI,0.617,0.350000,1,1.1,2.1,.524,3.2,6.0,...,0.0,0.6,1.4,8.2,.778,PG,29,82,0,22.4
404,John Stockton*,UTA,0.607,0.291667,1,1.2,2.8,.449,5.2,9.6,...,0.3,3.3,2.6,14.7,.804,PG,32,82,82,35.0
389,Kenny Smith,HOU,0.604,0.561644,1,1.8,4.1,.429,3.5,7.3,...,0.1,1.5,1.3,10.4,.851,PG,29,81,81,25.1
95,Hubert Davis,NYK,0.586,0.466667,1,1.6,3.5,.455,3.6,7.5,...,0.1,1.1,1.8,10.0,.808,SG,24,82,4,20.7
309,Shaquille O'Neal*,ORL,0.583,0.004950,0,0.0,0.1,.000,11.8,20.2,...,2.4,2.6,3.3,29.3,.533,C,22,79,79,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,Darrick Martin,MIN,0.423,0.159420,0,0.2,1.1,.184,2.8,6.9,...,0.0,1.8,2.6,7.5,.877,PG,23,34,9,23.6
168,Tom Gugliotta,WSB,0.422,0.094203,0,0.7,1.3,.500,5.5,13.8,...,1.8,2.5,3.2,16.0,.788,PF,25,6,6,37.7
393,Steve Smith,MIA,0.414,0.413793,1,1.0,6.0,.167,5.5,14.5,...,0.5,2.0,4.5,20.5,.773,SG,25,2,2,31.0
265,Donyell Marshall,MIN,0.411,0.254717,1,0.8,2.7,.302,4.0,10.6,...,1.3,1.5,1.6,10.8,.680,SF,21,40,8,25.9
